# Some tensorflow basics

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## Tensor

In [2]:
t0 = tf.constant(3, dtype=tf.int32)

In [3]:
t0

<tf.Tensor 'Const:0' shape=() dtype=int32>

In [4]:
t1 = tf.constant([3, 4.1, 5.2], dtype=tf.float32)

In [5]:
t1

<tf.Tensor 'Const_1:0' shape=(3,) dtype=float32>

In [6]:
t2 = tf.constant([['Apple', 'Orange'], ['Patato', 'Tomato']], dtype=tf.string)

In [7]:
t2

<tf.Tensor 'Const_2:0' shape=(2, 2) dtype=string>

In [8]:
t3 = tf.constant([[[5],[6],[7]],[[4],[5],[6]]])

In [9]:
t3

<tf.Tensor 'Const_3:0' shape=(2, 3, 1) dtype=int32>

## Dataflow Graph
A data-flow graph (DFG) is a graph which represents a data dependancies between a number of operations. Each node defines an operation, where inputs and outputs are tensor.

## Session

A session allows to execute graphs or part of graphs. It allocates resources (on one or more machines) for that and holds the actual values of intermediate results and variables.

## Construct a computation graph

### A simple add operation

In [10]:
node1 = tf.constant(3.2)
node2 = tf.constant(4.8)
adder = node1+node2
adder

<tf.Tensor 'add:0' shape=() dtype=float32>

In [11]:
sess = tf.Session()
sess.run(adder)

8.0

### Use placeholder to customize input

In [12]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a+b
adder

<tf.Tensor 'add:0' shape=() dtype=float32>

In [13]:
sess = tf.Session()
sess.run(adder_node, {a: 3, b: 4.5})

7.5

In [14]:
sess.run(adder_node, {a: [2, 3], b:[3, 4]})

array([ 5.,  7.], dtype=float32)

### Add additional computations

In [15]:
add_and_triple = adder_node*3
sess.run(add_and_triple, {a: 3, b: 4})

21.0

## Linear Model With Manual Fix

In [16]:
W = tf.Variable([.1], dtype=tf.float32)
b = tf.Variable([.1], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x+b

In [17]:
y = tf.placeholder(tf.float32)
loss = tf.reduce_sum(tf.square(linear_model-y))

In [18]:
sess = tf.Session()

tf.Variable() needs initialization

In [19]:
init = tf.global_variables_initializer()
sess.run(init)

In [20]:
sess.run(W)

array([ 0.1], dtype=float32)

In [21]:
sess.run(b)

array([ 0.1], dtype=float32)

Fitted value

In [22]:
sess.run(linear_model, {x: [1, 2, 3, 6, 8]})

array([ 0.2       ,  0.30000001,  0.40000001,  0.70000005,  0.90000004], dtype=float32)

Loss function

In [23]:
sess.run(loss, {x: [1, 2, 3, 6, 8], y: [4.8, 8.5, 10.4, 21.0, 25.3]})

1195.85

Reassign values manually

In [24]:
fixW = tf.assign(W, [2.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])

[array([ 2.], dtype=float32), array([ 1.], dtype=float32)]

In [25]:
sess.run(loss, {x: [1, 2, 3, 6, 8], y: [4.8, 8.5, 10.4, 21.0, 25.3]})

159.93997

## Train models

In [26]:
x_train = [1, 2, 3, 6, 8]
y_train = [4.8, 8.5, 10.4, 21.0, 25.3]

In [27]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
train = optimizer.minimize(loss)

Train 10000 times

In [28]:
for i in range(10000):
    sess.run(train, {x: x_train, y: y_train})

In [29]:
print('W: %s b: %s loss: %s' % (sess.run(W), sess.run(b), 
                                sess.run(loss, {x: x_train , y: y_train})))

W: [ 2.98236108] b: [ 2.07054377] loss: 2.12941


## Linear Regressor model

In [30]:
feature_columns = [tf.feature_column.numeric_column('x', shape = [1])]
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/zd/3xysxg0s077_yl8jw9xyzrv40000gq/T/tmp75jqr7f3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c29f65978>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
x_train = np.array([1., 2., 3., 6., 8.])
y_train = np.array([4.8, 8.5, 10.4, 21.0, 25.3])
x_test = np.array([2., 5., 7., 9.])
y_test = np.array([7.6, 17.2, 23.6, 28.8])

In [32]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'x': x_train}, y_train, batch_size=2, num_epochs=None, shuffle=True)
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'x': x_train}, y_train, batch_size=2, num_epochs=1000, shuffle=False)
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_test}, y_test, batch_size=2, num_epochs=1000, shuffle=False)

Train 1000 times

In [33]:
estimator.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/zd/3xysxg0s077_yl8jw9xyzrv40000gq/T/tmp75jqr7f3/model.ckpt.
INFO:tensorflow:loss = 663.13, step = 1
INFO:tensorflow:global_step/sec: 857.45
INFO:tensorflow:loss = 18.1281, step = 101 (0.119 sec)
INFO:tensorflow:global_step/sec: 1081.86
INFO:tensorflow:loss = 1.07167, step = 201 (0.093 sec)
INFO:tensorflow:global_step/sec: 1067.86
INFO:tensorflow:loss = 0.757966, step = 301 (0.094 sec)
INFO:tensorflow:global_step/sec: 1056.4
INFO:tensorflow:loss = 1.5059, step = 401 (0.095 sec)
INFO:tensorflow:global_step/sec: 1063.8
INFO:tensorflow:loss = 1.37246, step = 501 (0.094 sec)
INFO:tensorflow:global_step/sec: 1092.69
INFO:tensorflow:loss = 0.667293, step = 601 (0.091 sec)
INFO:tensorflow:global_step/sec: 1074.73
INFO:tensorflow:loss = 2.35976, step = 701 (0.094 sec)
INFO:tensorflow:global_step/sec: 1056.66
INFO:tensorflow:loss = 1.13394, step = 801 (0.094 sec)
INFO:tensorflow:global_step/sec

In [34]:
train_metrics = estimator.evaluate(input_fn=val_input_fn)
train_metrics

INFO:tensorflow:Starting evaluation at 2018-11-26-17:05:51
INFO:tensorflow:Restoring parameters from /var/folders/zd/3xysxg0s077_yl8jw9xyzrv40000gq/T/tmp75jqr7f3/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-11-26-17:05:53
INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.516148, global_step = 1000, loss = 1.0323


{'average_loss': 0.51614815, 'global_step': 1000, 'loss': 1.0322963}

In [35]:
test_metrics = estimator.evaluate(input_fn=test_input_fn)
test_metrics

INFO:tensorflow:Starting evaluation at 2018-11-26-17:05:53
INFO:tensorflow:Restoring parameters from /var/folders/zd/3xysxg0s077_yl8jw9xyzrv40000gq/T/tmp75jqr7f3/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-11-26-17:05:55
INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.369772, global_step = 1000, loss = 0.739543


{'average_loss': 0.36977169, 'global_step': 1000, 'loss': 0.73954338}

## Customized Estimator

In [36]:
def model_fn(features, labels, mode):
    W = tf.get_variable('W', [1], dtype=tf.float64)
    b = tf.get_variable('b', [1], dtype=tf.float64)
    y = W*features['x']+b
    
    loss = tf.reduce_sum(tf.square(y-labels))
    
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))
    
    return tf.estimator.EstimatorSpec(mode=mode, predictions=y, loss=loss, train_op=train)

In [37]:
estimator = tf.estimator.Estimator(model_fn=model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/zd/3xysxg0s077_yl8jw9xyzrv40000gq/T/tmp3355wqz2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c2b0302e8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [38]:
estimator.train(input_fn=train_input_fn, steps = 1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/zd/3xysxg0s077_yl8jw9xyzrv40000gq/T/tmp3355wqz2/model.ckpt.
INFO:tensorflow:loss = 90.7219982269, step = 1
INFO:tensorflow:global_step/sec: 923.463
INFO:tensorflow:loss = 0.0861456345626, step = 101 (0.111 sec)
INFO:tensorflow:global_step/sec: 1079.13
INFO:tensorflow:loss = 3.1452762197, step = 201 (0.092 sec)
INFO:tensorflow:global_step/sec: 1087.07
INFO:tensorflow:loss = 0.126975145636, step = 301 (0.092 sec)
INFO:tensorflow:global_step/sec: 726.464
INFO:tensorflow:loss = 0.183317377507, step = 401 (0.137 sec)
INFO:tensorflow:global_step/sec: 1090.56
INFO:tensorflow:loss = 0.0426468169276, step = 501 (0.092 sec)
INFO:tensorflow:global_step/sec: 1084.01
INFO:tensorflow:loss = 7.12410727118, step = 601 (0.092 sec)
INFO:tensorflow:global_step/sec: 1080.44
INFO:tensorflow:loss = 0.176827105669, step = 701 (0.093 sec)
INFO:tensorflow:global_step/sec: 1083.29
INFO:tensorflow:loss = 2.3896

In [39]:
train_metrics = estimator.evaluate(input_fn=val_input_fn)

INFO:tensorflow:Starting evaluation at 2018-11-26-17:05:56
INFO:tensorflow:Restoring parameters from /var/folders/zd/3xysxg0s077_yl8jw9xyzrv40000gq/T/tmp3355wqz2/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-11-26-17:05:58
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.924095


In [40]:
train_metrics

{'global_step': 1000, 'loss': 0.92409474}

In [41]:
test_metrics = estimator.evaluate(input_fn=test_input_fn)
test_metrics

INFO:tensorflow:Starting evaluation at 2018-11-26-17:05:58
INFO:tensorflow:Restoring parameters from /var/folders/zd/3xysxg0s077_yl8jw9xyzrv40000gq/T/tmp3355wqz2/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-11-26-17:06:00
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.307932


{'global_step': 1000, 'loss': 0.30793205}

## TensorBoard

In [42]:
linear_model

<tf.Tensor 'add_2:0' shape=<unknown> dtype=float32>

In [43]:
with tf.name_scope('loss-model'):
    loss = tf.reduce_sum(tf.square(linear_model - y))
    tf.summary.scalar('loss', loss)

In [44]:
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

In [45]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [46]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./TensorBoard_files/LinearModel', sess.graph)

In [47]:
for i in range(10000):
    summary, _ = sess.run([merged, train], {x: x_train, y: y_train})
    writer.add_summary(summary, i)

In [48]:
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})

In [49]:
print("After train W: %s b %s loss: %s" % (curr_W, curr_b, curr_loss))

After train W: [ 2.98236108] b [ 2.07054377] loss: 2.12941


TensorBoard can be visualized by `tensorboard --logdir .`

## A neural Network example

In [50]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Hyperparameters

In [51]:
learning_rate = 0.5
epochs = 10
batch_size = 100

Input image has 28*28 = 784 pixels

Output is one-hot of 10 digits

In [52]:
x = tf.placeholder(tf.float32, [None, 784], name='x-input')
y = tf.placeholder(tf.float32, [None, 10], name='y-input')

In [53]:
# hidden layer
W1 = tf.Variable(tf.random_normal([784, 300], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([300]), name='b1')
# output layer
W2 = tf.Variable(tf.random_normal([300, 10], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([10]), name='b2')

Hidden network

In [54]:
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

Output

In [55]:
y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))

Define loss function

In [56]:
y_clipped = tf.clip_by_value(y_, 1e-10, 1-1e-7)
cross_entropy = -tf.reduce_mean(
    tf.reduce_sum(y*tf.log(y_clipped)+(1-y_clipped)*tf.log(1-y_clipped),
                 axis = 1))

Optimizer

In [57]:
optimizer = tf.train.GradientDescentOptimizer(
    learning_rate = learning_rate).minimize(cross_entropy)

Initialize

In [58]:
init = tf.global_variables_initializer()
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Train model

In [59]:
with tf.Session() as sess:
    sess.run(init)
    total_batch = int(len(mnist.train.labels)/batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            _, c = sess.run([optimizer, cross_entropy], {x: batch_x, y: batch_y})
            avg_cost += c/total_batch
        print('Epoch:', (epoch+1), 'cost = %.3f' %avg_cost)
    print('Accuracy after all:', sess.run(accuracy, {x: mnist.test.images, y: mnist.test.labels}))

Epoch: 1 cost = 0.527
Epoch: 2 cost = 0.201
Epoch: 3 cost = 0.139
Epoch: 4 cost = 0.108
Epoch: 5 cost = 0.085
Epoch: 6 cost = 0.069
Epoch: 7 cost = 0.057
Epoch: 8 cost = 0.046
Epoch: 9 cost = 0.037
Epoch: 10 cost = 0.030
Accuracy after all: 0.9797
